### Problem Statement

Develop a predictive framework for enabling proactive retension strategy for a Telecom Company.

### Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Importing the Data for ML project

In [ ]:
df = pd.read_csv('/Users/priyankac/Downloads/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
# Checking the number of rows and colums in dataset
df.shape

In [ ]:
# Checking the first 5 records
df.head()

In [ ]:
# Checking different datatypes in the given dataset
df.dtypes

### Setting Display options to ensure feature name visiblity

In [ ]:
pd.set_option('display.max_columns', None)

### Warning Suppression

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### How many rows have missing ID?

In [ ]:
df['customerID'].isnull().sum()

### Drop ID feature from the dataset

In [ ]:
df = df.drop(['customerID'], axis = 1)

In [ ]:
df.dtypes

### Label the Churn feature to 1/0

In [ ]:
# Getting count of 'yes' and 'no' in Churn
df['Churn'].value_counts()

In [ ]:
df['target'] = np.where(df['Churn'] == 'Yes',1,0 )

### Drop the Churn feature to retain only target

In [ ]:
df = df.drop(['Churn'], axis = 1)

In [ ]:
df.dtypes

### Defining Target and Independent features

In [ ]:
Y = df[['target']]

In [ ]:
X = df.drop(['target'], axis = 1)

### Get the Churn rate

In [ ]:
Y.mean()

### Split features into Numerical and Categorical

In [ ]:
num = X.select_dtypes(include = 'number')
char = X.select_dtypes(include = 'object')

In [ ]:
num.tail()

In [ ]:
num.describe()

In [ ]:
# Check whether SeniorCitizen feature is an indicator
num.SeniorCitizen.value_counts()

In [ ]:
char.head()

### Dropping the indicator feature from num to build a separate DF

In [ ]:
ind = num[['SeniorCitizen']]
num = num.drop(['SeniorCitizen'], axis = 1)

In [ ]:
num.dtypes

In [ ]:
ind.dtypes

In [ ]:
char.dtypes

### Outlier Analysis of Numerical Features

In [ ]:
num.describe(percentiles = [0.01,0.05,0.10,0.25,0.50,0.75,0.85,0.9,0.99])

### Capping and Flooring of outliers

In [ ]:
def outlier_cap(x):
    x = x.clip(lower = x.quantile(0.01))
    x = x.clip(upper = x.quantile(0.99))
    return(x)

In [ ]:
num = num.apply(lambda x : outlier_cap(x))

In [ ]:
num.describe(percentiles = [0.01,0.05,0.10,0.25,0.50,0.75,0.85,0.9,0.99])

### Missing Values Analysis

In [ ]:
num.isnull().sum()

In [ ]:
# Since the data does not contain any missing values Imputation is not required
# X = X.loc[:, X.isnull().mean <= .25]

## Feature Selection - Numerical Features

### Part 1 : Remove Features with 0 Variance

In [ ]:
from sklearn.feature_selection import VarianceThreshold

varselector = VarianceThreshold(threshold = 0)
varselector.fit_transform(num)

# Get columns to keep and create new dataframe with those only
cols = varselector.get_support(indices = True)
num_1 = num.iloc[: , cols]

In [ ]:
num_1.iloc[0]

### Part 2 : Bi Variate Analysis (Feature Discretization)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

discrete = KBinsDiscretizer(n_bins = 10, encode = 'ordinal', strategy = 'quantile')

# type(discrete.fit_transform(num_1))

num_binned = pd.DataFrame(discrete.fit_transform(num_1), index = num_1.index,
                          columns = num_1.columns).add_suffix('_Rank')

num_binned.head()

In [ ]:
# Check if features show  a slope at all
# If they do, then do you see some deciles below the population average and some higher than the population average?
# If that is the case then the slope will be strong

# Conclusion: A strong slope is indicative of the features' ability to discriminate the event from non event
#              making it a good preductor

X_bin_combined = pd.concat([Y, num_binned], axis = 1, join = 'inner')

from numpy import mean
for col in (num_binned.columns):
    plt.figure()
    sns.lineplot(x=col, y=X_bin_combined['target'].mean(), data=X_bin_combined, color='red')
    sns.barplot(x=col, y='target', data=X_bin_combined, estimator=mean)
plt.show()    
    

In [ ]:
# All features from num_1 will get selected due to good discrimination power by all of them
select_features_df_num = num_1

In [ ]:
select_features_df_num.shape

## Feature Selection - Categorical Features


### Part 1 : Bi Variate Analysis

In [ ]:
X_char_merged = pd.concat([Y, char], axis = 1, join = 'inner')

from numpy import mean

for col in (char.columns):
    plt.figure()
    sns.lineplot(x=col, y=X_char_merged['target'].mean(), data=X_char_merged, color = 'red')
    sns.barplot(x=col, y='target', data=X_char_merged, estimator = mean)
plt.show()    

In [ ]:
# Drop gender, PhoneService and MultipleLines feature
char = char.drop(['gender', 'PhoneService', 'MultipleLines'], axis = 1)

In [ ]:
# Create dummy features with n-1 levels
X_char_dum = pd.get_dummies(char, drop_first = True)
X_char_dum.shape

### Part 2 : Select K Best

In [ ]:
# Select K Best for Categorical features

from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k = 20)
selector.fit_transform(X_char_dum, Y)

# Select columns to keep and create new dataframe with those only
cols = selector.get_support(indices = True)
select_features_df_char = X_char_dum.iloc[: , cols]

In [ ]:
select_features_df_char.shape

### Feature Selection - Numerical Indicator Feature

In [ ]:
# Feature selection for SeniorCitizen feature
X_ind_merged = pd.concat([Y, ind], axis = 1, join = 'inner')

from numpy import mean
for col in (ind.columns):
    plt.figure()
    sns.barplot(x = col, y ='target', data = X_ind_merged, estimator = mean)
plt.show()    

In [ ]:
select_features_df_ind = ind

### Creating the Master Feature Set for Model Development

In [ ]:
X_all = pd.concat([select_features_df_char, select_features_df_num, select_features_df_ind], axis = 1, join = 'inner')

In [ ]:
X_all.shape

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, Y, test_size = 0.3, random_state = 99)

In [ ]:
print('Shape of Training Data : ', X_train.shape)
print('Shape of Testing Data : ', X_test.shape)
print('Churn Rate in Training Data : ', y_train.mean())
print('Churn Rate in Testing Data : ', y_test.mean())

### Model Building Step

In [ ]:
# Building Logistic Regression model
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state = 99)
logreg.fit(X_train, y_train)

In [ ]:
coeff_df=pd.DataFrame(X_all.columns)
coeff_df.columns=['features']
coeff_df["Coefficient Estimate"] = pd.Series(logreg.coef_[0])
coeff_df

In [ ]:
# Building a Decision Tree Model
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(criterion = 'gini', random_state = 99)

In [ ]:
# Using GridSearchCV to find the best parameters

np.random.seed(44)
from sklearn.model_selection import GridSearchCV
param_dist = {'max_depth': [3, 5, 6, 7], 'min_samples_split': [50, 100, 150, 200,250]}
tree_grid = GridSearchCV(dtree, cv = 10, param_grid = param_dist, n_jobs = -1)
tree_grid.fit(X_train, y_train)
print('Best parameters using grid search: \n', tree_grid.best_params_)

In [ ]:
dtree = DecisionTreeClassifier(criterion = 'gini', random_state = 99, max_depth = 5, min_samples_split = 50)
dtree.fit(X_train, y_train)

In [ ]:
!pip install pydotplus

In [ ]:
from sklearn import tree
import pydotplus

plt.figure(figsize = [50, 10])
tree.plot_tree(dtree, filled = True, fontsize = 15, rounded = True, feature_names = X_all.columns)
plt.show()

In [ ]:
# Building Random Forest Model
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(criterion = 'gini', random_state = 99, max_depth = 5, min_samples_split = 50)
rf.fit(X_train, y_train)

In [ ]:
# Getting feature importances
import pandas as pd
feature_importances = pd.DataFrame(rf.feature_importances_,
                                  index = X_train.columns,
                                  columns = ['importance']).sort_values('importance', ascending = False)
feature_importances

In [ ]:
# Building the Gradient Boosting Model
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(criterion = 'mse', random_state = 99, max_depth = 5, min_samples_split = 50)
gbm.fit(X_train, y_train)

In [ ]:
feature_importances = pd.DataFrame(gbm.feature_importances_,
                                  index = X_train.columns,
                                  columns = ['importance']).sort_values('importance', ascending = False)
feature_importances

In [ ]:
# Building StackingClassifier
base_learners = [
        ('rf', RandomForestClassifier(criterion='gini',random_state=99, max_depth=5,min_samples_split=50)),
        ('gbm', GradientBoostingClassifier(criterion='mse',random_state=99,max_depth=5,min_samples_split=50))
        ]

In [ ]:
from sklearn.ensemble import StackingClassifier

clf = StackingClassifier(estimators = base_learners, final_estimator = LogisticRegression())

In [ ]:
clf.fit(X_train, y_train)

### Model Evaluation

In [ ]:

# For Logistic regression
y_pred_logreg = logreg.predict(X_test)

# For Decison tree
y_pred_tree = dtree.predict(X_test)

# For Random forest
y_pred_rf = rf.predict(X_test)

# For Gradient boosting
y_pred_gbm = gbm.predict(X_test)

# For Stacking Classifier
y_pred_clf = clf.predict(X_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
# For Logistic Regression Model

print('Accuracy : ',metrics.accuracy_score(y_test, y_pred_logreg))
print('Precision : ',metrics.precision_score(y_test, y_pred_logreg))
print('Recall : ',metrics.recall_score(y_test, y_pred_logreg))
print('f1_score : ',metrics.f1_score(y_test, y_pred_logreg))

In [ ]:
# Plotting Confusion matrix
metrics.plot_confusion_matrix(logreg, X_test, y_test)

In [ ]:
# For Decision Tree

print('Accuracy : ',metrics.accuracy_score(y_test, y_pred_tree))
print('Precision : ',metrics.precision_score(y_test, y_pred_tree))
print('Recall : ',metrics.recall_score(y_test, y_pred_tree))
print('f1_score : ',metrics.f1_score(y_test, y_pred_tree))

In [ ]:
# Plotting Confusion matrix
metrics.plot_confusion_matrix(dtree, X_test, y_test)

In [ ]:
# For Random Forest model

print('Accuracy : ',metrics.accuracy_score(y_test, y_pred_rf))
print('Precision : ',metrics.precision_score(y_test, y_pred_rf))
print('Recall : ',metrics.recall_score(y_test, y_pred_rf))
print('f1_score : ',metrics.f1_score(y_test, y_pred_rf))

In [ ]:
# Plotting Confusion matrix
metrics.plot_confusion_matrix(rf, X_test, y_test)

In [ ]:
# For Gradient Boosting Model

print('Accuracy : ',metrics.accuracy_score(y_test, y_pred_gbm))
print('Precision ; ',metrics.precision_score(y_test, y_pred_gbm))
print('Recall : ',metrics.recall_score(y_test, y_pred_gbm))
print('f1_score : ',metrics.f1_score(y_test, y_pred_gbm))


In [ ]:
# plotting Confusion matrix
metrics.plot_confusion_matrix(gbm, X_test, y_test)

In [ ]:
# For StackingClassifier

print('Accuracy : ',metrics.accuracy_score(y_test, y_pred_clf))
print('Precision : ',metrics.precision_score(y_test, y_pred_clf))
print('Recall : ',metrics.recall_score(y_test, y_pred_clf))
print('f1_score : ',metrics.f1_score(y_test, y_pred_clf))

In [ ]:
# Plotting Confusion matrix
metrics.plot_confusion_matrix(clf, X_test, y_test)

### Lorenze Curve

In [ ]:
# Logistic Regression Lorenze Curve

y_pred_prob = logreg.predict_proba(X_all)[:, 1]
df['pred_prob_logreg'] = pd.DataFrame(y_pred_prob)
df['P_Rank_logreg'] = pd.qcut(df['pred_prob_logreg'].rank(method='first').values, 10, duplicates='drop').codes+1
rank_df_actuals = df.groupby('P_Rank_logreg')['target'].agg(['count', 'mean'])
rank_df_actuals.rename(columns = {'mean': 'Actual_event_rate'}, inplace = True)
rank_df_actuals


In [ ]:
# Sorting the rank_df_actuals in descending order
sorted_rank_df = rank_df_actuals.sort_values(by = 'P_Rank_logreg', ascending = False)
sorted_rank_df

In [ ]:
# After sorting the data we find number of people who actually churned
sorted_rank_df['N_events'] = sorted_rank_df['count'] * sorted_rank_df['Actual_event_rate']
sorted_rank_df

In [ ]:
# finding the cummulative event
sorted_rank_df['cum_events'] = sorted_rank_df['N_events'].cumsum()
sorted_rank_df

In [ ]:
# finding the event capture
sorted_rank_df['event_cap'] = sorted_rank_df['N_events']/max(sorted_rank_df['N_events'].cumsum())
sorted_rank_df

In [ ]:
# Finding cummulative event capture
sorted_rank_df['cum_event_cap'] = sorted_rank_df['event_cap'].cumsum()
sorted_rank_df

In [ ]:
# Finding data for non events

sorted_rank_df['N_non_events'] = sorted_rank_df['count'] - sorted_rank_df['N_events']
sorted_rank_df

In [ ]:
# Cummulative non event
sorted_rank_df['cum_non_events'] = sorted_rank_df['N_non_events'].cumsum()
sorted_rank_df

In [ ]:
# Finding non event capture rate
sorted_rank_df['non_event_cap'] = sorted_rank_df['N_non_events']/max(sorted_rank_df['N_non_events'].cumsum())
sorted_rank_df


In [ ]:
# finding cummulative non event capture rate
sorted_rank_df['cum_non_event_cap'] = sorted_rank_df['non_event_cap'].cumsum()
sorted_rank_df

In [ ]:
# Using KS-The KS test report the maximum difference between the two cumulative distributions, 
# and calculates a P value from that and the sample sizes.

sorted_rank_df['KS'] = round((sorted_rank_df['cum_event_cap'] - sorted_rank_df['cum_non_event_cap']), 4)
sorted_rank_df

# resetting the index
sorted_reindexed = sorted_rank_df.reset_index()
sorted_reindexed['Decile'] = sorted_reindexed.index+1
sorted_reindexed


In [ ]:
# plotting a graph
ax = sns.lineplot(x = 'Decile', y = 'cum_event_cap', data = sorted_reindexed, color = 'grey')
ax = sns.lineplot(x = 'Decile', y = 'cum_non_event_cap', data = sorted_reindexed, color = 'red')


In [ ]:
ax = sns.barplot(x = 'Decile', y = 'KS', data = sorted_reindexed, color = 'blue')
# at Decile 4 the KS value is maximum

In [ ]:
# Project conclusion:
# The Logistic regression model has perfromed the best and will be used for targeting customer 
# with retension offers in Telecom

In [ ]:
# Creating rank on the feature tenure
df['Tenure_Rank'] = pd.qcut(df['tenure'].rank(method = 'first').values , 10, duplicates = 'drop').codes+1

In [ ]:
df.groupby('Tenure_Rank')['tenure'].agg(['min', 'max', 'mean'])

In [ ]:
df['tenure'].mean()

In [ ]:
# Mean of tenure is 32. In the above Tenure_Rank we separate anything above rank 6 as high tenure and below 
# as low tenure(this is decided on the value that is closest to the tenure mean)

In [ ]:
df['Tenure_Segment'] = np.where(df['Tenure_Rank'] <= 6, 'Low Tenure', 'High Tenure')

In [ ]:
# Creating rank for the MonthlyCharges feature and finding the low charges and high charges customers
df['MonthlyCharge_Rank'] = pd.qcut(df['MonthlyCharges'].rank(method='first').values, 10, duplicates = 'drop').codes+1

In [ ]:
df.groupby('MonthlyCharge_Rank')['MonthlyCharges'].agg(['min', 'max', 'mean'])

In [ ]:
df['MonthlyCharges'].mean()

In [ ]:
# Based on the mean(64), any customer below rank 5 can be considered as low charges and above as high charges
df['Monthly_Charge_Segment'] = np.where(df['MonthlyCharge_Rank'] <= 5, 'Low Charges', 'High Charges')

In [ ]:
df['Predicted_Churn_Rank'] = np.where(df['P_Rank_logreg'] >= 7, 'Top 4', 'Bottom 6')

In [ ]:
df.head()

### Slice the data with respect to 'Top 4' from the Logistic Model Output

In [ ]:
df_top4 = df.loc[df['Predicted_Churn_Rank'] == 'Top 4', :]

In [ ]:
service_list = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
               'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling']

In [ ]:
df_top4_services = df_top4[service_list]

In [ ]:
# Finding the services used most by the customers in the top 4 list
for col in (df_top4_services.columns):
    plt.figure()
    sns.countplot(x = col, data = df_top4_services)
plt.show()    

In [ ]:
# Top 4 are the customers most likely to churn (concluded from Logistic Regression Model).So whatever 
# services they are using the most are the problematic services.

In [ ]:
pd.crosstab(index = df_top4['Monthly_Charge_Segment'], columns = df_top4['Tenure_Segment'],
            values = df_top4['MonthlyCharges'], aggfunc = 'mean')

In [ ]:
pd.crosstab(index = df_top4['Monthly_Charge_Segment'], columns = df_top4['Tenure_Segment'],
           values = df_top4['target'], aggfunc = 'count')

In [ ]:
### Recommendations###

# The insights coming from the model is as such:
# Device Protection with Online services should be offered
# Convert customer to DSL if they are facing challenges with Fibre Optics
# Offer discounts on yearly contracts